In [ ]:
import os
print("Current working directory:", os.getcwd())

In [ ]:
#!/usr/bin/env python3
"""
Run Simple Parallel Scan for Test Bar

This script runs the test bar scan across multiple exchanges in parallel
using a simplified parallel scanning approach that avoids console output issues.
"""

import asyncio
import sys
import os
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(message)s')

# Add project directory to path
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)
print(f"✓ Added {project_dir} to sys.path")

# Add current directory to path
sys.path.append(os.getcwd())
print(f"✓ Added {os.getcwd()} to sys.path")

# Import the simple parallel scanner
from run_parallel_scanner import run_parallel_exchanges, run_parallel_multi_timeframes_all_exchanges
from scanner.main import kline_cache

# Define exchanges
futures_exchanges = ["binance_futures", "bybit_futures", "mexc_futures", "gateio_futures"]
spot_exchanges = ["binance_spot", "bybit_spot", "kucoin_spot", "mexc_spot", "gateio_spot"]
spot_exchanges_1w = ["binance_spot", "bybit_spot", "gateio_spot"]

async def main():
    # Clear cache for fresh data
    kline_cache.clear()
    
    """
    # Run parallel scan for test bar strategy on spot exchanges
    result = await run_parallel_exchanges(
        timeframe="4h",                    # Example timeframe
        strategies=["confluence", "test_bar", "consolidation"],
        # strategies=["reversal_bar"],       
        exchanges=spot_exchanges,          # Spot exchanges to scan
        users=["default"],                 # Recipients for Telegram notifications
        send_telegram=True,                # Enable Telegram notifications
        min_volume_usd=None                # Use default volume threshold
    )
    """
    # Run multi-timeframe parallel scan
    result = await run_parallel_multi_timeframes_all_exchanges(
        timeframes=["2d"],     # Multiple timeframes
        strategies=["hbs_breakout"],        # Strategies to scan
        exchanges=["binance_spot", "binance_futures", "bybit_spot", "kucoin_spot", "mexc_spot", "gateio_spot"],          # Exchanges to scan
        users=["default"],                 # Recipients for notifications
        send_telegram=True,                # Enable notifications
        min_volume_usd=None                # Use default volume threshold
    )
     #"""
    
    print("Scan completed!")
    return result

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
#!/usr/bin/env python3
"""
Seven Figures Weekly Strategy Scanner for KuCoin & MEXC

This notebook runs weekly strategy detection using SF server data for KuCoin and MEXC exchanges.
You can choose between: hbs_breakout, consolidation_breakout, and confluence strategies.
Both exchanges are scanned simultaneously with parallel processing.
"""

import asyncio
import sys
import os
import pandas as pd
import numpy as np
import logging
from datetime import datetime
from tqdm.asyncio import tqdm
import nest_asyncio

# Setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
nest_asyncio.apply()

# Add project directory to path
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)
print(f"✓ Added {project_dir} to sys.path")

# Import strategies and SF service
from exchanges.sf_pairs_service import SFPairsService
from custom_strategies import detect_consolidation_breakout, detect_confluence

class SFWeeklyStrategyScanner:
    """
    Weekly strategy scanner using Seven Figures server data
    """
    
    def __init__(self):
        self.sf_service = SFPairsService()
        self.strategy_functions = {
            'consolidation_breakout': self.detect_consolidation_breakout,
            'confluence': self.detect_confluence,
            'hbs_breakout': self.detect_hbs_breakout
        }
        
    def prepare_sf_data(self, raw_df):
        """Convert SF data to strategy-compatible format"""
        if raw_df is None or len(raw_df) == 0:
            return None
        
        df = pd.DataFrame(raw_df)
        
        # Convert datetime column to pandas datetime and set as index
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'])
            df = df.set_index('datetime')
        elif 'time' in df.columns:
            df['time'] = pd.to_datetime(df['time'], unit='ms')
            df = df.set_index('time')
        
        # Select only OHLCV columns needed
        required_cols = ['open', 'high', 'low', 'close', 'volume']
        available_cols = [col for col in required_cols if col in df.columns]
        
        if len(available_cols) != 5:
            return None
        
        # Select and clean data
        result_df = df[required_cols].copy()
        
        # Ensure numeric types
        for col in required_cols:
            result_df[col] = pd.to_numeric(result_df[col], errors='coerce')
        
        # Drop any NaN rows
        result_df = result_df.dropna()
        
        # Sort by index (oldest first)
        result_df = result_df.sort_index()
        
        return result_df
    
    def detect_consolidation_breakout(self, df, symbol, exchange):
        """Detect consolidation breakout signals"""
        signals = []
        
        # Check last closed bar
        if len(df) > 1:
            detected, result = detect_consolidation_breakout(df, check_bar=-2)
            if detected:
                # Calculate volume info
                volume_usd = df['volume'].iloc[-2] * df['close'].iloc[-2]
                volume_mean = df['volume'].rolling(7).mean().iloc[-2]
                volume_ratio = df['volume'].iloc[-2] / volume_mean if volume_mean > 0 else 0
                
                signals.append({
                    'symbol': symbol,
                    'exchange': exchange,
                    'strategy': 'consolidation_breakout',
                    'direction': result.get('direction'),
                    'timestamp': result.get('timestamp', df.index[-2]),
                    'close': df['close'].iloc[-2],
                    'current_bar': False,
                    'volume_usd': volume_usd,
                    'volume_ratio': volume_ratio,
                    'box_hi': result.get('box_hi'),
                    'box_lo': result.get('box_lo'),
                    'box_age': result.get('box_age'),
                    'bars_inside': result.get('bars_inside'),
                    'height_pct': result.get('height_pct'),
                })
        
        # Check current bar
        if len(df) > 2:
            detected, result = detect_consolidation_breakout(df, check_bar=-1)
            if detected:
                # Calculate volume info
                volume_usd = df['volume'].iloc[-1] * df['close'].iloc[-1]
                volume_mean = df['volume'].rolling(7).mean().iloc[-1]
                volume_ratio = df['volume'].iloc[-1] / volume_mean if volume_mean > 0 else 0
                
                signals.append({
                    'symbol': symbol,
                    'exchange': exchange,
                    'strategy': 'consolidation_breakout',
                    'direction': result.get('direction'),
                    'timestamp': result.get('timestamp', df.index[-1]),
                    'close': df['close'].iloc[-1],
                    'current_bar': True,
                    'volume_usd': volume_usd,
                    'volume_ratio': volume_ratio,
                    'box_hi': result.get('box_hi'),
                    'box_lo': result.get('box_lo'),
                    'box_age': result.get('box_age'),
                    'bars_inside': result.get('bars_inside'),
                    'height_pct': result.get('height_pct'),
                })
        
        return signals
    
    def detect_confluence(self, df, symbol, exchange):
        """Detect confluence signals"""
        signals = []
        
        # Check last closed bar
        if len(df) > 1:
            detected, result = detect_confluence(df, check_bar=-2)
            if detected:
                signals.append({
                    'symbol': symbol,
                    'exchange': exchange,
                    'strategy': 'confluence',
                    'timestamp': result['timestamp'],
                    'close': result['close_price'],
                    'current_bar': False,
                    'volume': result['volume'],
                    'volume_usd': result['volume_usd'],
                    'volume_ratio': result['volume_ratio'],
                    'close_off_low': result['close_off_low'],
                    'momentum_score': result['momentum_score'],
                    'high_volume': result['high_volume'],
                    'spread_breakout': result['spread_breakout'],
                    'momentum_breakout': result['momentum_breakout'],
                })
        
        # Check current bar
        if len(df) > 2:
            detected, result = detect_confluence(df, check_bar=-1)
            if detected:
                signals.append({
                    'symbol': symbol,
                    'exchange': exchange,
                    'strategy': 'confluence',
                    'timestamp': result['timestamp'],
                    'close': result['close_price'],
                    'current_bar': True,
                    'volume': result['volume'],
                    'volume_usd': result['volume_usd'],
                    'volume_ratio': result['volume_ratio'],
                    'close_off_low': result['close_off_low'],
                    'momentum_score': result['momentum_score'],
                    'high_volume': result['high_volume'],
                    'spread_breakout': result['spread_breakout'],
                    'momentum_breakout': result['momentum_breakout'],
                })
        
        return signals
    
    def detect_hbs_breakout(self, df, symbol, exchange):
        """Detect HBS (Consolidation + Confluence) breakout signals"""
        signals = []
        
        # Check both bars for consolidation breakout
        cb_detected_prev, cb_result_prev = detect_consolidation_breakout(df, check_bar=-2)
        cb_detected_curr, cb_result_curr = detect_consolidation_breakout(df, check_bar=-1)
        
        # Check both bars for confluence
        cf_detected_prev, cf_result_prev = detect_confluence(df, check_bar=-2)
        cf_detected_curr, cf_result_curr = detect_confluence(df, check_bar=-1)
        
        # HBS requires BOTH consolidation breakout AND confluence
        cb_fired = cb_detected_prev or cb_detected_curr
        cf_fired = cf_detected_prev or cf_detected_curr
        
        if cb_fired and cf_fired:
            # Use the most recent consolidation breakout result
            if cb_detected_curr:
                cb_result = cb_result_curr
                bar_idx = -1
                current_bar = True
            else:
                cb_result = cb_result_prev
                bar_idx = -2
                current_bar = False
            
            # Calculate volume info
            volume_usd = df['volume'].iloc[bar_idx] * df['close'].iloc[bar_idx]
            volume_mean = df['volume'].rolling(7).mean().iloc[bar_idx]
            volume_ratio = df['volume'].iloc[bar_idx] / volume_mean if volume_mean > 0 else 0
            
            signals.append({
                'symbol': symbol,
                'exchange': exchange,
                'strategy': 'hbs_breakout',
                'direction': cb_result.get('direction'),
                'timestamp': cb_result.get('timestamp', df.index[bar_idx]),
                'close': df['close'].iloc[bar_idx],
                'current_bar': current_bar,
                'volume_usd': volume_usd,
                'volume_ratio': volume_ratio,
                
                # Consolidation data
                'box_hi': cb_result.get('box_hi'),
                'box_lo': cb_result.get('box_lo'),
                'box_age': cb_result.get('box_age'),
                'bars_inside': cb_result.get('bars_inside'),
                'height_pct': cb_result.get('height_pct'),
                
                # Confluence data
                'cf_detected_prev': cf_detected_prev,
                'cf_detected_curr': cf_detected_curr,
                'cb_detected_prev': cb_detected_prev,
                'cb_detected_curr': cb_detected_curr,
            })
        
        return signals
    
    async def scan_exchange_pairs(self, exchange, strategy, volume_threshold=300000, max_pairs=None):
        """Scan all pairs for an exchange with the selected strategy"""
        print(f"\n🔍 Scanning {exchange} for {strategy} signals...")
        print(f"📊 Volume threshold: ${volume_threshold:,}")
        
        # Get all pairs from exchange
        all_pairs = self.sf_service.get_pairs_of_exchange(exchange)
        
        # Filter for USDT pairs
        usdt_pairs = [
            pair for pair in all_pairs 
            if 'Quote' in pair and pair['Quote'].upper() == "USDT"
        ]
        
        print(f"📋 Found {len(usdt_pairs)} USDT pairs on {exchange}")
        
        # Use all pairs unless max_pairs is specified
        if max_pairs is None:
            pairs_to_scan = usdt_pairs
            print(f"🔄 Scanning ALL {len(pairs_to_scan)} pairs...")
        else:
            pairs_to_scan = usdt_pairs[:max_pairs]
            print(f"🔄 Scanning {len(pairs_to_scan)} pairs (limited by max_pairs={max_pairs})...")
        
        all_signals = []
        
        # Use tqdm for progress tracking
        for pair in tqdm(pairs_to_scan, desc=f"{exchange} {strategy}"):
            try:
                token = pair['Token']
                
                # Skip stablecoins
                if token.upper() in ['USDT', 'USDC', 'BUSD', 'DAI', 'TUSD']:
                    continue
                
                # Get weekly OHLCV data (50 weeks for good analysis)
                raw_data = self.sf_service.get_ohlcv_for_pair(
                    token, 'USDT', exchange, '1w', 50
                )
                
                if raw_data is None or len(raw_data) == 0:
                    continue
                
                # Prepare data
                df = self.prepare_sf_data(raw_data)
                if df is None or len(df) < 25:  # Need minimum data
                    continue
                
                # Check volume threshold on recent bars (both current and last closed)
                current_volume_usd = df['close'].iloc[-1] * df['volume'].iloc[-1]
                last_closed_volume_usd = df['close'].iloc[-2] * df['volume'].iloc[-2]
                
                # Accept if either current or last closed bar meets volume threshold
                if current_volume_usd < volume_threshold and last_closed_volume_usd < volume_threshold:
                    continue
                
                # Run the selected strategy
                strategy_func = self.strategy_functions.get(strategy)
                if strategy_func:
                    signals = strategy_func(df, f"{token}USDT", exchange)
                    if signals:
                        all_signals.extend(signals)
                        for signal in signals:
                            bar_type = "current" if signal.get('current_bar') else "last closed"
                            direction = signal.get('direction', 'N/A')
                            print(f"🎯 {strategy} signal found: {token}USDT on {exchange} ({bar_type} bar, {direction})")
                
            except Exception as e:
                # Skip errors silently to avoid spam
                continue
        
        print(f"✅ {exchange} scan complete: {len(all_signals)} {strategy} signals found")
        return all_signals
    
    async def run_parallel_scan(self, strategy='consolidation_breakout', volume_threshold=300000, max_pairs=None):
        """Run parallel scan on both KuCoin and MEXC"""
        
        exchanges = ['Kucoin', 'Mexc']
        strategy_name = strategy.replace('_', ' ').title()
        
        print(f"🚀 STARTING PARALLEL SF WEEKLY SCAN")
        print("="*80)
        print(f"📊 Strategy: {strategy_name}")
        print(f"🏢 Exchanges: {', '.join(exchanges)}")
        print(f"📅 Timeframe: Weekly (1w)")
        print(f"💰 Min Volume: ${volume_threshold:,}")
        if max_pairs:
            print(f"🎯 Max Pairs per Exchange: {max_pairs}")
        else:
            print(f"🎯 Scanning ALL pairs on each exchange")
        print(f"📊 Checking: Current bar AND last closed bar")
        print(f"🕐 Start Time: {datetime.now().strftime('%H:%M:%S')}")
        print("="*80)
        
        start_time = datetime.now()
        
        # Create tasks for both exchanges
        tasks = [
            self.scan_exchange_pairs(exchange, strategy, volume_threshold, max_pairs)
            for exchange in exchanges
        ]
        
        # Run both exchanges in parallel
        results = await asyncio.gather(*tasks)
        
        # Combine results
        all_signals = []
        for exchange_signals in results:
            all_signals.extend(exchange_signals)
        
        end_time = datetime.now()
        duration = end_time - start_time
        
        # Display results
        print(f"\n{'='*80}")
        print(f"📊 SCAN RESULTS - {strategy_name}")
        print(f"{'='*80}")
        print(f"🕐 Duration: {str(duration).split('.')[0]}")
        print(f"🎯 Total Signals: {len(all_signals)}")
        
        if all_signals:
            # Convert to DataFrame for better display
            df_results = pd.DataFrame(all_signals)
            
            # Group by exchange
            for exchange in exchanges:
                exchange_signals = [s for s in all_signals if s['exchange'] == exchange]
                print(f"\n📈 {exchange}: {len(exchange_signals)} signals")
            
            # Sort by volume for display
            df_results = df_results.sort_values('volume_usd', ascending=False)
            
            # Display key columns based on strategy
            if strategy == 'consolidation_breakout':
                display_cols = ['symbol', 'exchange', 'direction', 'close', 'volume_usd', 
                               'volume_ratio', 'box_age', 'bars_inside', 'height_pct']
            elif strategy == 'confluence':
                display_cols = ['symbol', 'exchange', 'close', 'volume_usd', 'volume_ratio', 
                               'momentum_score', 'high_volume', 'spread_breakout', 'momentum_breakout']
            elif strategy == 'hbs_breakout':
                display_cols = ['symbol', 'exchange', 'direction', 'close', 'volume_usd', 
                               'volume_ratio', 'box_age', 'cf_detected_prev', 'cf_detected_curr']
            
            # Filter available columns
            available_cols = [col for col in display_cols if col in df_results.columns]
            
            print(f"\n📋 DETAILED RESULTS:")
            print("-" * 120)
            print(df_results[available_cols].to_string(index=False))
            
            # Show some statistics
            if 'direction' in df_results.columns:
                direction_counts = df_results['direction'].value_counts()
                print(f"\n📊 Direction Breakdown:")
                for direction, count in direction_counts.items():
                    print(f"  {direction}: {count} signals")
            
            print(f"\n💰 Volume Statistics:")
            print(f"  Average Volume: ${df_results['volume_usd'].mean():,.0f}")
            print(f"  Max Volume: ${df_results['volume_usd'].max():,.0f}")
            print(f"  Min Volume: ${df_results['volume_usd'].min():,.0f}")
            
        else:
            print(f"\n❌ No {strategy_name} signals found")
            print("💡 Try:")
            print("   • Lowering volume threshold")
            print("   • Increasing max_pairs")
            print("   • Different strategy")
        
        return all_signals

# Create scanner instance
scanner = SFWeeklyStrategyScanner()

# =============================================================================
# MAIN EXECUTION FUNCTIONS
# =============================================================================

async def scan_consolidation_breakout(volume_threshold=300000, max_pairs=None):
    """Scan for consolidation breakout signals"""
    return await scanner.run_parallel_scan(
        strategy='consolidation_breakout',
        volume_threshold=volume_threshold,
        max_pairs=max_pairs
    )

async def scan_confluence(volume_threshold=300000, max_pairs=None):
    """Scan for confluence signals"""
    return await scanner.run_parallel_scan(
        strategy='confluence',
        volume_threshold=volume_threshold,
        max_pairs=max_pairs
    )

async def scan_hbs_breakout(volume_threshold=300000, max_pairs=None):
    """Scan for HBS (hybrid) breakout signals"""
    return await scanner.run_parallel_scan(
        strategy='hbs_breakout',
        volume_threshold=volume_threshold,
        max_pairs=max_pairs
    )

# =============================================================================
# EASY-TO-USE INTERFACE
# =============================================================================

print("🎯 SF WEEKLY STRATEGY SCANNER FOR KUCOIN & MEXC")
print("="*60)
print("Available strategies:")
print("1. await scan_consolidation_breakout()  # Consolidation breakout detection")
print("2. await scan_confluence()              # Multi-factor confluence signals")  
print("3. await scan_hbs_breakout()            # Hybrid breakout (consolidation + confluence)")
print()
print("Optional parameters:")
print("• volume_threshold: Minimum USD volume (default: 300,000)")
print("• max_pairs: Max pairs per exchange (default: None = ALL pairs)")
print()
print("Examples:")
print("• await scan_consolidation_breakout()                    # Scan ALL pairs")
print("• await scan_hbs_breakout(volume_threshold=500000)       # Higher volume threshold")
print("• await scan_confluence(max_pairs=100)                   # Limit to 100 pairs per exchange")
print("• await scan_hbs_breakout(volume_threshold=200000)       # Lower volume threshold")

# Uncomment one of these to auto-run:
# await scan_consolidation_breakout()
# await scan_confluence() 
await scan_hbs_breakout()

In [ ]:
# Binance BTC dominated pairs - Confluence Scanner with Direct API

from telegram.ext import Application
import logging
import pandas as pd
import numpy as np
import asyncio
import nest_asyncio
from datetime import datetime
from tqdm.asyncio import tqdm
import sys
import os
import html
import aiohttp
import time

# Add project path
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)

from custom_strategies import detect_confluence

class BinanceBTCConfluenceScanner:
    def __init__(self, telegram_token, telegram_chat_id, timeframe, offset=1):
        self.telegram_token = telegram_token
        self.telegram_chat_id = telegram_chat_id
        self.telegram_app = None
        self.exchange = "Binance"
        self.timeframe = timeframe
        self.quote_currency = "BTC"
        self.offset = offset
        self.base_url = "https://api.binance.com"
        self.session = None
        
        # Binance timeframe mapping - All available intervals
        self.timeframe_map = {
            # Minutes
            "1m": "1m",
            "3m": "3m", 
            "5m": "5m",
            "15m": "15m",
            "30m": "30m",
            # Hours
            "1h": "1h",
            "2h": "2h",
            "4h": "4h",
            "6h": "6h",
            "8h": "8h",
            "12h": "12h",
            # Days
            "1d": "1d",
            "2d": "2d",
            "3d": "3d",
            # Weeks/Months
            "1w": "1w",
            "1M": "1M"
        }
        
    async def init_session(self):
        """Initialize aiohttp session"""
        if self.session is None:
            self.session = aiohttp.ClientSession()
            
    async def close_session(self):
        """Close aiohttp session"""
        if self.session:
            await self.session.close()
            self.session = None
        
    async def init_telegram(self):
        if self.telegram_app is None:
            self.telegram_app = Application.builder().token(self.telegram_token).build()

    async def get_btc_pairs(self):
        """Get all BTC trading pairs from Binance"""
        await self.init_session()
        
        try:
            url = f"{self.base_url}/api/v3/exchangeInfo"
            async with self.session.get(url) as response:
                if response.status == 200:
                    data = await response.json()
                    
                    # Filter for BTC pairs that are actively trading
                    btc_pairs = []
                    for symbol_info in data['symbols']:
                        if (symbol_info['quoteAsset'] == 'BTC' and 
                            symbol_info['status'] == 'TRADING' and
                            symbol_info['isSpotTradingAllowed']):
                            
                            btc_pairs.append({
                                'symbol': symbol_info['symbol'],
                                'baseAsset': symbol_info['baseAsset'],
                                'quoteAsset': symbol_info['quoteAsset']
                            })
                    
                    return btc_pairs
                else:
                    logging.error(f"Error fetching exchange info: {response.status}")
                    return []
        except Exception as e:
            logging.error(f"Error fetching BTC pairs: {str(e)}")
            return []

    async def get_klines(self, symbol, interval, limit=100):
        """Get kline/candlestick data from Binance"""
        await self.init_session()
        
        try:
            url = f"{self.base_url}/api/v3/klines"
            params = {
                'symbol': symbol,
                'interval': interval,
                'limit': limit
            }
            
            async with self.session.get(url, params=params) as response:
                if response.status == 200:
                    data = await response.json()
                    
                    # Convert to DataFrame
                    df = pd.DataFrame(data, columns=[
                        'open_time', 'open', 'high', 'low', 'close', 'volume',
                        'close_time', 'quote_asset_volume', 'number_of_trades',
                        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
                    ])
                    
                    # Convert timestamps to datetime
                    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
                    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
                    
                    # Convert OHLCV to numeric
                    for col in ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume']:
                        df[col] = pd.to_numeric(df[col], errors='coerce')
                    
                    # Set datetime index
                    df.set_index('open_time', inplace=True)
                    
                    # Select only OHLCV columns needed for confluence
                    # Note: Using quote_asset_volume as it's the volume in BTC
                    result_df = df[['open', 'high', 'low', 'close', 'quote_asset_volume']].copy()
                    result_df.rename(columns={'quote_asset_volume': 'volume'}, inplace=True)
                    
                    return result_df
                    
                elif response.status == 429:
                    # Rate limit hit, wait a bit
                    await asyncio.sleep(1)
                    return None
                else:
                    return None
                    
        except Exception as e:
            logging.error(f"Error fetching klines for {symbol}: {str(e)}")
            return None

    async def send_telegram_alert(self, results):
        if not results:
            return
            
        try:
            message = f"🚨 Confluence Detection - Binance BTC Pairs {self.timeframe}\n\n"
            
            # Map timeframe to TradingView format - Extended mapping
            tv_timeframe_map = {
                # Minutes
                "1m": "1", "3m": "3", "5m": "5", "15m": "15", "30m": "30",
                # Hours  
                "1h": "60", "2h": "120", "4h": "240", "6h": "360", "8h": "480", "12h": "720",
                # Days
                "1d": "1D", "2d": "2D", "3d": "3D",
                # Weeks/Months
                "1w": "1W", "1M": "1M"
            }
            tv_timeframe = tv_timeframe_map.get(self.timeframe.lower(), self.timeframe)
            
            for result in results:
                formatted_symbol = result['symbol']
                tv_link = f"https://www.tradingview.com/chart/?symbol=BINANCE:{formatted_symbol}&interval={tv_timeframe}"
                
                # Escape HTML entities in the URL
                escaped_link = html.escape(tv_link)
                
                # Format according to BTC specifications
                time_str = ""
                if result.get('timestamp') is not None:
                    time_str = f"Time: {result['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}\n"
                
                message += (
                    f"Symbol: {result['symbol']}\n"
                    f"{time_str}"
                    f"Volume BTC: ₿{result['volume_btc']:,.4f}\n"
                    f"Close: <a href='{escaped_link}'>₿{result['close']:.8f}</a>\n"
                    f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                    f"Close Off Low: {result['close_off_low']:.1f}%\n"
                    f"Momentum: {result['momentum_score']:.4f}\n"
                    f"{'='*30}\n"
                )
            
            # Split message more carefully to avoid breaking HTML tags
            max_length = 4000
            
            if len(message) > max_length:
                # Split at natural breaks (between results) to avoid breaking HTML
                sections = message.split('='*30 + '\n')
                current_chunk = ""
                
                for section in sections:
                    if len(current_chunk + section + '='*30 + '\n') > max_length:
                        if current_chunk:
                            await self.telegram_app.bot.send_message(
                                chat_id=self.telegram_chat_id,
                                text=current_chunk.strip(),
                                parse_mode='HTML',
                                disable_web_page_preview=True
                            )
                        current_chunk = section + '\n'
                    else:
                        current_chunk += section + '='*30 + '\n'
                
                # Send remaining chunk
                if current_chunk.strip():
                    await self.telegram_app.bot.send_message(
                        chat_id=self.telegram_chat_id,
                        text=current_chunk.strip(),
                        parse_mode='HTML',
                        disable_web_page_preview=True
                    )
            else:
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=message,
                    parse_mode='HTML',
                    disable_web_page_preview=True
                )
                
        except Exception as e:
            logging.error(f"Error sending Telegram alert: {str(e)}")
            
            # Fallback: send without HTML formatting
            try:
                simple_message = f"🚨 Confluence Detection - Binance BTC Pairs {self.timeframe}\n\n"
                for result in results:
                    simple_message += (
                        f"Symbol: {result['symbol']}\n"
                        f"Volume BTC: ₿{result['volume_btc']:,.4f}\n"
                        f"Close: ₿{result['close']:.8f}\n"
                        f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                        f"Components: Vol={result['high_volume']}, Spread={result['spread_breakout']}, Mom={result['momentum_breakout']}\n\n"
                    )
                
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=simple_message,
                    disable_web_page_preview=True
                )
            except Exception as fallback_error:
                logging.error(f"Fallback Telegram send also failed: {str(fallback_error)}")

    def scan_single_market(self, pair, df):
        """Scan a single market for Confluence pattern in the specified bar"""
        try:
            if df is None or len(df) < 50:  # Need enough data for confluence
                return None
            
            # Calculate which bar to check based on offset
            check_bar = -(self.offset + 1)  # offset=0 means current bar (-1), offset=1 means last closed (-2), etc.
            
            # Run confluence detection
            detected, result = detect_confluence(df, check_bar=check_bar)
            
            if detected:
                # Get the target bar values
                target_close = df['close'].iloc[check_bar]
                target_volume = df['volume'].iloc[check_bar]  # This is already in BTC
                
                confluence_result = {
                    'symbol': pair['symbol'],
                    'volume_btc': float(target_volume),
                    'close': float(target_close),
                    'volume': float(target_volume),
                    'volume_ratio': result['volume_ratio'],
                    'close_off_low': result['close_off_low'],
                    'momentum_score': result['momentum_score'],
                    'high_volume': result['high_volume'],
                    'spread_breakout': result['spread_breakout'],
                    'momentum_breakout': result['momentum_breakout'],
                    'bar_range': result['bar_range'],
                    'timestamp': df.index[check_bar] if hasattr(df.index, '__getitem__') else None
                }
                return confluence_result
                
        except Exception as e:
            logging.error(f"Error processing {pair['symbol']}: {str(e)}")
        return None

    async def scan_all_markets(self):
        """Scan all BTC markets for Confluence pattern"""
        await self.init_telegram()
        
        try:
            # Define volume thresholds for BTC pairs based on timeframe
            volume_thresholds = {
                # Minutes - higher volume needed for shorter timeframes
                "1m": 0.01,   "3m": 0.02,   "5m": 0.03,
                "15m": 0.05,  "30m": 0.08,
                # Hours
                "1h": 0.1,    "2h": 0.15,   "4h": 0.2,
                "6h": 0.25,   "8h": 0.3,    "12h": 0.35,
                # Days
                "1d": 0.4,    "2d": 0.8,    "3d": 1.2,
                # Weeks/Months
                "1w": 2.0,    "1M": 8.0
            }
            min_volume = volume_thresholds.get(self.timeframe.lower(), 0.1)  # Default 0.1 BTC
            
            # Create offset description
            if self.offset == 0:
                offset_desc = "current candle"
            elif self.offset == 1:
                offset_desc = "last closed candle"
            else:
                offset_desc = f"{self.offset} candles ago"
            
            print(f"Scanning Binance BTC pairs for Confluence patterns in {offset_desc}...")
            print(f"Timeframe: {self.timeframe}")
            print(f"Minimum volume threshold: ₿{min_volume:.2f}")
            
            # Get all BTC pairs from Binance
            print("Fetching BTC pairs from Binance...")
            btc_pairs = await self.get_btc_pairs()
            
            if not btc_pairs:
                print("No BTC pairs found or error fetching pairs")
                return []
            
            print(f"Found {len(btc_pairs)} BTC pairs to scan")
            
            # Filter out stablecoins and obvious non-trading pairs
            filtered_pairs = []
            skip_tokens = ['USDT', 'USDC', 'BUSD', 'DAI', 'TUSD', 'USDD', 'FDUSD']
            
            for pair in btc_pairs:
                if pair['baseAsset'] not in skip_tokens:
                    filtered_pairs.append(pair)
            
            print(f"After filtering: {len(filtered_pairs)} pairs to scan")
            
            # Get Binance timeframe - fallback to 1d if not found
            binance_interval = self.timeframe_map.get(self.timeframe.lower(), "1d")
            
            # Process all pairs with progress bar
            all_results = []
            successful_scans = 0
            
            with tqdm(total=len(filtered_pairs), desc="Scanning markets") as pbar:
                for pair in filtered_pairs:
                    try:
                        # Get OHLCV data from Binance
                        df = await self.get_klines(pair['symbol'], binance_interval, 100)
                        
                        if df is None or len(df) < 50:
                            pbar.update(1)
                            continue
                        
                        successful_scans += 1
                        target_idx = -(self.offset + 1)
                        
                        # Update progress bar with current symbol
                        pbar.set_description(f"Scanning: {pair['symbol']} ({len(df)} candles)")
                        
                        # Check volume threshold for the target candle
                        try:
                            target_candle_volume = float(df['volume'].iloc[target_idx])  # Already in BTC
                            
                            # Only process if volume meets threshold
                            if target_candle_volume >= min_volume:
                                result = self.scan_single_market(pair, df)
                                if result:
                                    all_results.append(result)
                                    print(f"Found Confluence: {pair['symbol']} 🎯")
                        except (IndexError, ValueError):
                            pass  # Skip if we can't calculate volume
                        
                        # Add small delay to respect rate limits
                        await asyncio.sleep(0.1)
                                
                    except Exception as e:
                        if "429" in str(e):
                            # Rate limit - add longer delay
                            await asyncio.sleep(2)
                    finally:
                        pbar.update(1)
            
            print(f"Successfully scanned {successful_scans}/{len(filtered_pairs)} pairs")
            
            # Sort by volume
            all_results.sort(key=lambda x: x['volume_btc'], reverse=True)
            
            # Send Telegram alert if we found any patterns
            if all_results:
                await self.send_telegram_alert(all_results)
            
            return all_results
            
        except Exception as e:
            logging.error(f"Error scanning markets: {str(e)}")
            return []
        finally:
            await self.close_session()

async def run_binance_btc_confluence_scanner(timeframe, offset=1):
    """
    Run the Binance BTC Confluence scanner
    
    Parameters:
    timeframe (str): Time period - Available options:
                    Minutes: 1m, 3m, 5m, 15m, 30m
                    Hours: 1h, 2h, 4h, 6h, 8h, 12h  
                    Days: 1d, 2d, 3d
                    Weeks/Months: 1w, 1M
    offset (int): Bar offset (0=current, 1=last closed, 2=two bars ago, etc.)
    """
    
    if offset == 0:
        offset_desc = "current candle"
    elif offset == 1:
        offset_desc = "last closed candle"
    else:
        offset_desc = f"{offset} candles ago"
    
    print(f"Starting Binance BTC Confluence scan for {offset_desc} on {timeframe}...")
    
    # Use the confluence telegram token
    telegram_token = "8066329517:AAHVr6kufZWe8UqCKPfmsRhSPleNlt_7G-g"
    telegram_chat_id = "375812423"
    
    scanner = BinanceBTCConfluenceScanner(telegram_token, telegram_chat_id, timeframe, offset)
    results = await scanner.scan_all_markets()
    
    if results:
        print(f"\nFound {len(results)} Confluence patterns:")
        
        # Convert results to DataFrame for console display
        df_results = pd.DataFrame(results)
        
        # Round numeric columns for BTC precision
        df_results['volume_btc'] = df_results['volume_btc'].round(4)
        df_results['close'] = df_results['close'].round(8)
        df_results['volume'] = df_results['volume'].round(4)
        df_results['volume_ratio'] = df_results['volume_ratio'].round(2)
        df_results['close_off_low'] = df_results['close_off_low'].round(1)
        df_results['momentum_score'] = df_results['momentum_score'].round(4)
        
        # Reorder columns for better display
        display_cols = ['symbol', 'close', 'volume_btc', 'volume_ratio', 'close_off_low', 
                       'momentum_score', 'high_volume', 'spread_breakout', 'momentum_breakout']
        available_cols = [col for col in display_cols if col in df_results.columns]
        
        # Display the results
        print(df_results[available_cols])
        
        # Show component analysis
        print(f"\n🔧 COMPONENT ANALYSIS:")
        vol_count = df_results['high_volume'].sum()
        spread_count = df_results['spread_breakout'].sum()
        momentum_count = df_results['momentum_breakout'].sum()
        
        print(f"High Volume signals: {vol_count}/{len(results)} ({vol_count/len(results)*100:.1f}%)")
        print(f"Spread Breakout signals: {spread_count}/{len(results)} ({spread_count/len(results)*100:.1f}%)")
        print(f"Momentum Breakout signals: {momentum_count}/{len(results)} ({momentum_count/len(results)*100:.1f}%)")
        
    else:
        print(f"\nNo Confluence patterns found in {offset_desc}")

# Set up logging
logging.basicConfig(level=logging.ERROR)

# Apply nest_asyncio to allow async operations in Jupyter
nest_asyncio.apply()

# Example usage functions
async def scan_binance_btc_current():
    """Scan current candle for confluence - Binance BTC pairs"""
    await run_binance_btc_confluence_scanner("1w", offset=0)

async def scan_binance_btc_closed():
    """Scan last closed candle for confluence - Binance BTC pairs"""
    await run_binance_btc_confluence_scanner("1w", offset=1)

async def scan_binance_btc_previous():
    """Scan two candles ago for confluence - Binance BTC pairs"""
    await run_binance_btc_confluence_scanner("1w", offset=2)

# Main execution function
async def main():
    """
    Main execution - modify parameters here
    """
    timeframe = "1w"  # 1d, 2d, 3d, 1w
    offset = 0        # 0 = current candle, 1 = last closed candle, 2 = two candles ago
    
    await run_binance_btc_confluence_scanner(timeframe, offset)

# Run the async main function
print("🔍 BINANCE BTC CONFLUENCE SCANNER")
print("=" * 40)
print("Available timeframes:")
print("• Minutes: 1m, 3m, 5m, 15m, 30m")
print("• Hours: 1h, 2h, 4h, 6h, 8h, 12h") 
print("• Days: 1d, 2d, 3d")
print("• Weeks/Months: 1w, 1M")
print("\nAvailable functions:")
print("• await main() - Run with default settings")
print("• await scan_binance_btc_current() - Scan current candle")
print("• await scan_binance_btc_closed() - Scan last closed candle")
print("• await scan_binance_btc_previous() - Scan two candles ago")
print("• await run_binance_btc_confluence_scanner('timeframe', offset) - Custom scan")
print("\nExamples:")
print("• await run_binance_btc_confluence_scanner('4h', 1)  # 4-hour last closed")
print("• await run_binance_btc_confluence_scanner('15m', 0) # 15-min current")
print("• await run_binance_btc_confluence_scanner('1M', 1)  # Monthly last closed")
print("This scanner uses REAL Binance BTC pair volumes!")

# Uncomment to auto-run:
await main()

In [ ]:
# Debug ohlcv data of any pair

import asyncio
import sys
import os
import logging
import pandas as pd

logging.basicConfig(level=logging.INFO, format='%(message)s')

project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)
print(f"✓ Added {project_dir} to sys.path")

from exchanges import BybitFuturesClient  # Ensure this matches your exchanges/__init__.py

async def test_fetch():
    client = BybitFuturesClient(timeframe="2d")
    await client.init_session()
    df = await client.fetch_klines("L3USDT")
    await client.close_session()
    if df is not None:
        print("2d Candles for L3:")
        print(df.tail(5))  # Last 5 weeks
        last_row = df.iloc[-1]
        volume_usd = last_row['volume'] * last_row['close']
        print(f"Last Week: volume_usd={volume_usd:.2f}, close={last_row['close']}, volume={last_row['volume']:.2f}")

# Run the async function directly in the notebook
await test_fetch()

In [ ]:
#Direct strategy debug of any pair on any exchange
import asyncio
import sys
import os
import logging
import pandas as pd
import numpy as np
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s: %(message)s')
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)
print(f"✓ Added {project_dir} to sys.path")
from exchanges import MexcSpotClient, BybitSpotClient, GateioSpotClient, KucoinSpotClient, BinanceSpotClient, BinanceFuturesClient, BybitFuturesClient
from custom_strategies import detect_volume_surge, detect_weak_uptrend, detect_pin_down
from breakout_vsa import vsa_detector, breakout_bar_vsa, stop_bar_vsa, reversal_bar_vsa, start_bar_vsa, loaded_bar_vsa, test_bar_vsa

async def test_strategy(exchange_client_class, timeframe, symbol, strategy_name):
    client = exchange_client_class(timeframe=timeframe)
    await client.init_session()
    df = await client.fetch_klines(symbol)
    await client.close_session()
    
    if df is None or len(df) < 10:
        print(f"No data fetched for {symbol} or insufficient data (< 10 bars)")
        return
    
    print(f"{timeframe} Candles for {symbol}:")
    print(df.tail(5))
    last_row = df.iloc[-1]
    volume_usd = last_row['volume'] * last_row['close']
    print(f"Last Bar: volume_usd={volume_usd:.2f}, close={last_row['close']}, volume={last_row['volume']:.2f}")
    
    # Different handling based on strategy type
    if strategy_name == "volume_surge":
        # Use detect_volume_surge directly
        detected, result = detect_volume_surge(df)
        
        print(f"\nVolume Surge Detection Results:")
        print(f"Detected: {detected}")
        
        if detected:
            print(f"\nVolume Surge Details:")
            print(f"  Date: {result['timestamp']}")
            print(f"  Close: ${result['close_price']:,.8f}")
            print(f"  Volume: {result['volume']:,.2f}")
            print(f"  Volume USD: ${result['volume_usd']:,.2f}")
            print(f"  Volume Ratio: {result['volume_ratio']:,.2f}x")
            print(f"  Score: {result['score']:,.2f}")
            print(f"  Price Extreme: {result['price_extreme']}")
    
    elif strategy_name == "pin_down":
        from custom_strategies import detect_pin_down
        detected, result = detect_pin_down(df)
        
        print(f"\nPin Down Detection Results:")
        print(f"Detected: {detected}")
        
        if detected:
            print(f"\nPin Down Details:")
            for key, value in result.items():
                if key != 'symbol':  # Skip symbol as we already know it
                    print(f"  {key}: {value}")
    
    elif strategy_name == "weak_uptrend":
        from custom_strategies import detect_weak_uptrend
        detected, result = detect_weak_uptrend(df)
        
        print(f"\nWeak Uptrend Detection Results:")
        print(f"Detected: {detected}")
        
        if detected:
            print(f"\nWeak Uptrend Details:")
            for key, value in result.items():
                if key != 'symbol':  # Skip symbol as we already know it
                    print(f"  {key}: {value}")
    
    else:
        # For VSA strategies, import the appropriate get_params
        if strategy_name == "reversal_bar":
            from breakout_vsa.strategies.reversal_bar import get_params
        elif strategy_name == "breakout_bar":
            from breakout_vsa.strategies.breakout_bar import get_params
        elif strategy_name == "loaded_bar":
            from breakout_vsa.strategies.loaded_bar import get_params
        elif strategy_name == "stop_bar":
            from breakout_vsa.strategies.stop_bar import get_params
        elif strategy_name == "start_bar":
            from breakout_vsa.strategies.start_bar import get_params
        else:
            print(f"Unknown strategy: {strategy_name}")
            return
        
        # Use vsa_detector with strategy-specific params
        params = get_params()
        condition, result = vsa_detector(df, params)
        
        strategy_display_name = strategy_name.replace('_vsa', '').replace('_', ' ').title()
        print(f"\n{strategy_display_name} Detection Results:")
        print(f"Current Bar (index -1): {condition.iloc[-1]}")
        if len(df) > 1:
            print(f"Last Closed Bar (index -2): {condition.iloc[-2]}")
        
        if condition.iloc[-1] or (len(df) > 1 and condition.iloc[-2]):
            detected_idx = -1 if condition.iloc[-1] else -2
            volume_mean = df['volume'].rolling(7).mean().iloc[detected_idx]
            bar_range = df['high'].iloc[detected_idx] - df['low'].iloc[detected_idx]
            close_off_low = (df['close'].iloc[detected_idx] - df['low'].iloc[detected_idx]) / bar_range * 100 if bar_range > 0 else 0
            volume_usd_detected = df['volume'].iloc[detected_idx] * df['close'].iloc[detected_idx]
            
            arctan_ratio = result['arctan_ratio'].iloc[detected_idx]  # From result DataFrame
            
            print(f"\nDetected at index {detected_idx} ({'Current' if detected_idx == -1 else 'Last Closed'} Bar):")
            print(f"  Date: {df.index[detected_idx]}")
            print(f"  Close: ${df['close'].iloc[detected_idx]:,.8f}")
            print(f"  Volume Ratio: {df['volume'].iloc[detected_idx] / volume_mean if volume_mean > 0 else 0:.2f}x")
            print(f"  {timeframe} Volume: ${volume_usd_detected:.2f}")
            print(f"  Close Off Low: {close_off_low:.1f}%")
            print(f"  Angular Ratio: {arctan_ratio:.2f}")

# Define the test case
exchange_client = GateioSpotClient
timeframe = "1w"
symbol = "PRCL_USDT"
strategy = "loaded_bar"
await test_strategy(exchange_client, timeframe, symbol, strategy)

In [ ]:
#zip the project

import shutil
import os

# Go to parent directory of your project
os.chdir("/home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025")

# Create the zip file (this will include everything inside 'hbs_2025')
shutil.make_archive("Project_VSA_2025_backup", 'zip', "Project")


In [ ]:
#!/usr/bin/env python3 -> Debug built weekly candles for mexc and kucoin 
import sys
import os
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)
print(f"✓ Added {project_dir} to sys.path")
import asyncio
import logging
import pandas as pd
import numpy as np
from exchanges.kucoin_client import KucoinClient
from breakout_vsa.core import calculate_start_bar

from scanner.main import kline_cache
kline_cache.clear()  # Clear cache for fresh data

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

async def debug_start_bar_detection():
    # Initialize client
    client = KucoinClient(timeframe="1w")
    await client.init_session()
    
    # Symbol to debug
    symbol = "TAO-USDT"
    
    try:
        # Fetch data
        df = await client.fetch_klines(symbol)
        
        if df is not None:
            print(f"Weekly candles for {symbol}:")
            print(df.tail())
            
            # Add intermediate calculations to see what's happening
            # This is a modified version of calculate_start_bar that adds debugging
            lookback = 5
            volume_lookback = 30
            volume_percentile = 50
            low_percentile = 75
            range_percentile = 75
            close_off_lows_percent = 50
            prev_close_range = 75
            
            # Calculate basic bar characteristics
            df['bar_range'] = df['high'] - df['low']
            df['volume_rank'] = df['volume'].rolling(lookback).apply(
                lambda x: sum(1.0 for val in x if val <= x[-1]) / len(x) * 100, 
                raw=True
            )
            
            # Calculate rolling values
            df['macro_low'] = df['low'].rolling(volume_lookback).min()
            df['macro_high'] = df['high'].rolling(volume_lookback).max()
            df['highest_high'] = df['high'].rolling(lookback).max()
            
            # Volume conditions
            df['volume_sma'] = df['volume'].rolling(volume_lookback).mean()
            df['volume_std'] = df['volume'].rolling(volume_lookback).std()
            df['excess_volume'] = df['volume'] > (df['volume_sma'] + 3.0 * df['volume_std'])
            
            # Range conditions
            df['range_sma'] = df['bar_range'].rolling(volume_lookback).mean()
            df['range_std'] = df['bar_range'].rolling(volume_lookback).std()
            df['excess_range'] = df['bar_range'] > (df['range_sma'] + 3.0 * df['range_std'])
            
            # Volume percentile condition
            def is_in_top_percent(series, length, percent):
                ranks = series.rolling(length).apply(
                    lambda x: sum(1.0 for val in x if val <= x[-1]) / len(x) * 100, 
                    raw=True
                )
                return ranks >= percent
            
            def is_in_bottom_percent(series, length, percent):
                ranks = series.rolling(length).apply(
                    lambda x: sum(1.0 for val in x if val <= x[-1]) / len(x) * 100, 
                    raw=True
                )
                return ranks <= percent
            
            # Volume conditions
            df['is_higher_volume'] = is_in_top_percent(df['volume'], lookback, volume_percentile)
            df['is_high_volume'] = (df['volume'] > 0.75 * df['volume_sma']) & (df['volume'] > df['volume'].shift(1))
            
            # Price action conditions
            df['has_higher_high'] = df['high'] > df['high'].shift(1)
            df['no_narrow_range'] = is_in_top_percent(df['bar_range'], lookback, range_percentile)
            
            # Low price condition
            df['is_in_the_lows'] = (
                (df['low'] - df['macro_low']).abs() < df['bar_range']
            ) | is_in_bottom_percent(df['low'], volume_lookback, low_percentile)
            
            # Close position conditions
            df['close_in_the_highs'] = (
                (df['close'] - df['low']) / df['bar_range']
            ) >= (close_off_lows_percent / 100)
            
            # Previous close distance condition
            df['far_prev_close'] = (
                (df['close'] - df['close'].shift(1)).abs() >=
                (df['bar_range'].shift(1) * (prev_close_range / 100))
            )
            
            # New highs condition
            df['new_highs'] = df['high'] >= 0.75 * df['highest_high']
            
            # Optional strength condition
            df['strong_close'] = df['close'] >= df['highest_high'].shift(1)
            
            # Now check the actual values for the last few bars
            last_rows = df.tail(3)
            
            print("\nAnalyzing last 3 bars:")
            for idx, row in last_rows.iterrows():
                print(f"\nBar at {idx.strftime('%Y-%m-%d')}:")
                print(f"  is_high_volume: {row['is_high_volume']}")
                print(f"  has_higher_high: {row['has_higher_high']}")
                print(f"  no_narrow_range: {row['no_narrow_range']}")
                print(f"  close_in_the_highs: {row['close_in_the_highs']}")
                print(f"  far_prev_close: {row['far_prev_close']}")
                print(f"  excess_range: {row['excess_range']}")
                print(f"  excess_volume: {row['excess_volume']}")
                print(f"  new_highs: {row['new_highs']}")
                print(f"  is_in_the_lows: {row['is_in_the_lows']}")
                print(f"  volume: {row['volume']}, volume_sma: {row['volume_sma']}")
                print(f"  bar_range: {row['bar_range']}, range_sma: {row['range_sma']}")
                
            # Run the original function to confirm
            start_bar_pattern = calculate_start_bar(df)
            print(f"\nFinal Start Bar detection result:")
            print(start_bar_pattern.tail(3))
            
    except Exception as e:
        print(f"Error in debug: {str(e)}")
    finally:
        await client.close_session()

# Replace the last part of your script with this:
if __name__ == "__main__":
    try:
        # For Jupyter/IPython environments
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.run(debug_start_bar_detection())
    except ImportError:
        # For regular Python environments
        asyncio.run(debug_start_bar_detection())

In [ ]:
#zip the project

import shutil
import os

# Go to parent directory of your project
os.chdir("/home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025")

# Create the zip file (this will include everything inside 'hbs_2025')
shutil.make_archive("Project_VSA_2025_backup", 'zip', "Project")

In [ ]:
# ARCHIVE - Confluence Scanner with bar offset

from telegram.ext import Application
import logging
import pandas as pd
import numpy as np
import asyncio
import logging
import nest_asyncio
from datetime import datetime
from tqdm.asyncio import tqdm
import sys
import os
import html

# Add project path
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)

from exchanges.sf_pairs_service import SFPairsService
from custom_strategies import detect_confluence

class ConfluenceScanner:
    def __init__(self, telegram_token, telegram_chat_id, exchange, timeframe, offset=1):
        self.telegram_token = telegram_token
        self.telegram_chat_id = telegram_chat_id
        self.telegram_app = None
        self.exchange = exchange
        self.timeframe = timeframe
        self.offset = offset  # Added offset parameter
        self.sf_service = SFPairsService()
        
    async def init_telegram(self):
        if self.telegram_app is None:
            self.telegram_app = Application.builder().token(self.telegram_token).build()

    async def send_telegram_alert(self, results):
        if not results:
            return
            
        try:
            message = f"🚨 Confluence Detection - {self.exchange} {self.timeframe}\n\n"
            
            # Map timeframe to TradingView format
            tv_timeframe_map = {
                "1d": "1D",
                "2d": "2D",
                "1w": "1W"
            }
            tv_timeframe = tv_timeframe_map.get(self.timeframe.lower(), self.timeframe)
            
            for result in results:
                exchange_name = self.exchange.upper()
                formatted_symbol = f"{result['symbol']}"
                tv_link = f"https://www.tradingview.com/chart/?symbol={exchange_name}:{formatted_symbol}&interval={tv_timeframe}"
                
                # Escape HTML entities in the URL
                escaped_link = html.escape(tv_link)
                
                # Format according to specified requirements
                time_str = ""
                if result.get('timestamp') is not None:
                    time_str = f"Time: {result['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}\n"
                
                message += (
                    f"Symbol: {result['symbol']}\n"
                    f"{time_str}"
                    f"Volume USD: ${result['volume_usd']:,.2f}\n"
                    f"Close: <a href='{escaped_link}'>${result['close']:,.8f}</a>\n"
                    f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                    f"Close Off Low: {result['close_off_low']:.1f}%\n"
                    f"{'='*30}\n"
                )
            
            # Split message more carefully to avoid breaking HTML tags
            max_length = 4000  # Reduced from 4096 to be safer
            
            if len(message) > max_length:
                # Split at natural breaks (between results) to avoid breaking HTML
                sections = message.split('='*30 + '\n')
                current_chunk = ""
                
                for section in sections:
                    if len(current_chunk + section + '='*30 + '\n') > max_length:
                        if current_chunk:
                            await self.telegram_app.bot.send_message(
                                chat_id=self.telegram_chat_id,
                                text=current_chunk.strip(),
                                parse_mode='HTML',
                                disable_web_page_preview=True
                            )
                        current_chunk = section + '\n'
                    else:
                        current_chunk += section + '='*30 + '\n'
                
                # Send remaining chunk
                if current_chunk.strip():
                    await self.telegram_app.bot.send_message(
                        chat_id=self.telegram_chat_id,
                        text=current_chunk.strip(),
                        parse_mode='HTML',
                        disable_web_page_preview=True
                    )
            else:
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=message,
                    parse_mode='HTML',
                    disable_web_page_preview=True
                )
                
        except Exception as e:
            logging.error(f"Error sending Telegram alert: {str(e)}")
            
            # Fallback: send without HTML formatting
            try:
                simple_message = f"🚨 Confluence Detection - {self.exchange} {self.timeframe}\n\n"
                for result in results:
                    simple_message += (
                        f"Symbol: {result['symbol']}\n"
                        f"Volume USD: ${result['volume_usd']:,.2f}\n"
                        f"Close: ${result['close']:,.8f}\n"
                        f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                        f"Components: Vol={result['high_volume']}, Spread={result['spread_breakout']}, Mom={result['momentum_breakout']}\n\n"
                    )
                
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=simple_message,
                    disable_web_page_preview=True
                )
            except Exception as fallback_error:
                logging.error(f"Fallback Telegram send also failed: {str(fallback_error)}")

    def prepare_sf_data(self, raw_df):
        """Convert SF data to confluence-compatible format"""
        if raw_df is None or len(raw_df) == 0:
            return None
        
        df = pd.DataFrame(raw_df)
        
        # Convert datetime column to pandas datetime and set as index
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'])
            df = df.set_index('datetime')
        elif 'time' in df.columns:
            # Convert Unix timestamp to datetime
            df['time'] = pd.to_datetime(df['time'], unit='ms')
            df = df.set_index('time')
        
        # Select only OHLCV columns needed for confluence
        required_cols = ['open', 'high', 'low', 'close', 'volume']
        available_cols = [col for col in required_cols if col in df.columns]
        
        if len(available_cols) != 5:
            return None
        
        # Select and clean data
        result_df = df[required_cols].copy()
        
        # Ensure numeric types
        for col in required_cols:
            result_df[col] = pd.to_numeric(result_df[col], errors='coerce')
        
        # Drop any NaN rows
        result_df = result_df.dropna()
        
        return result_df

    def scan_single_market(self, pair, ohlcv_data):
        """Scan a single market for Confluence pattern in the specified bar"""
        try:
            # Prepare data for confluence analysis
            df = self.prepare_sf_data(ohlcv_data)
            
            if df is None or len(df) < 50:  # Need enough data for confluence
                return None
            
            # Calculate which bar to check based on offset
            check_bar = -(self.offset + 1)  # offset=0 means current bar (-1), offset=1 means last closed (-2), etc.
            
            # Run confluence detection
            detected, result = detect_confluence(df, check_bar=check_bar)
            
            if detected:
                # Calculate volume in USD for the target bar
                target_close = df['close'].iloc[check_bar]
                target_volume = df['volume'].iloc[check_bar]
                volume_usd = float(target_close) * float(target_volume)
                
                confluence_result = {
                    'symbol': f"{pair['Token']}{pair['Quote']}",
                    'volume_usd': volume_usd,
                    'close': float(target_close),
                    'volume': float(target_volume),
                    'volume_ratio': result['volume_ratio'],
                    'close_off_low': result['close_off_low'],
                    'momentum_score': result['momentum_score'],
                    'high_volume': result['high_volume'],
                    'spread_breakout': result['spread_breakout'],
                    'momentum_breakout': result['momentum_breakout'],
                    'bar_range': result['bar_range']
                }
                return confluence_result
                
        except Exception as e:
            logging.error(f"Error processing {pair['Token']}{pair['Quote']}: {str(e)}")
        return None

    async def scan_all_markets(self):
        """Scan all markets for Confluence pattern"""
        await self.init_telegram()
        try:
            # Define volume thresholds
            volume_thresholds = {
                "1w": 300000,
                "2d": 100000,
                "1d": 50000
            }
            min_volume = volume_thresholds.get(self.timeframe.lower(), 50000)
            
            # Create offset description
            if self.offset == 0:
                offset_desc = "current candle"
            elif self.offset == 1:
                offset_desc = "last closed candle"
            else:
                offset_desc = f"{self.offset} candles ago"
            
            print(f"Scanning for Confluence patterns in {offset_desc}...")
            print(f"Minimum volume threshold: ${min_volume:,.0f}")
            
            # Get all pairs from SF service
            pairs = self.sf_service.get_pairs_of_exchange(self.exchange)
            print(f"Found {len(pairs)} markets to scan...")
            
            # Process all pairs with progress bar
            all_results = []
            with tqdm(total=len(pairs), desc="Scanning markets") as pbar:
                for pair in pairs:
                    try:
                        # Get OHLCV data from SF service
                        ohlcv_data = self.sf_service.get_ohlcv_for_pair(
                            pair['Token'], 
                            pair['Quote'], 
                            self.exchange, 
                            self.timeframe, 
                            100  # Get more data for confluence analysis
                        )
                        
                        if ohlcv_data is None or len(ohlcv_data) == 0:
                            pbar.update(1)
                            continue
                        
                        df = pd.DataFrame(ohlcv_data)
                        
                        # Check if we have enough data
                        if len(df) >= 50:  # Need enough for confluence analysis
                            target_idx = -(self.offset + 1)  # Adjust index based on offset
                            
                            # Check volume threshold for the target candle
                            try:
                                target_candle_volume = float(df['close'].iloc[target_idx]) * float(df['volume'].iloc[target_idx])
                                
                                # Only process if volume meets threshold
                                if target_candle_volume >= min_volume:
                                    result = self.scan_single_market(pair, ohlcv_data)
                                    if result:
                                        all_results.append(result)
                                        print(f"Found Confluence: {pair['Token']}{pair['Quote']} 🎯")
                            except (IndexError, ValueError):
                                pass  # Skip if we can't calculate volume
                                    
                    except Exception as e:
                        if "500" not in str(e):  # Don't log 500 errors
                            logging.error(f"Error processing {pair['Token']}{pair['Quote']}: {str(e)}")
                    finally:
                        pbar.update(1)
            
            # Sort by volume
            all_results.sort(key=lambda x: x['volume_usd'], reverse=True)
            
            # Send Telegram alert if we found any patterns
            if all_results:
                await self.send_telegram_alert(all_results)
            
            return all_results
            
        except Exception as e:
            logging.error(f"Error scanning markets: {str(e)}")
            return []

async def run_confluence_scanner(exchange, timeframe, offset=1):
    """
    Run the Confluence scanner
    
    Parameters:
    exchange (str): Exchange name (Kucoin, Mexc, Binance)
    timeframe (str): Time period (1d, 2d, 1w)
    offset (int): Bar offset (0=current, 1=last closed, 2=two bars ago, etc.)
    """
    
    if offset == 0:
        offset_desc = "current candle"
    elif offset == 1:
        offset_desc = "last closed candle"
    else:
        offset_desc = f"{offset} candles ago"
    
    print(f"Starting Confluence scan for {offset_desc} on {exchange} {timeframe}...")
    
    # Use the confluence telegram token from your big project config
    # You should replace this with the actual token from utils/config.py TELEGRAM_TOKENS["confluence"]
    telegram_token = "8066329517:AAHVr6kufZWe8UqCKPfmsRhSPleNlt_7G-g"  # Replace with confluence token
    telegram_chat_id = "375812423"  # Your chat ID
    
    scanner = ConfluenceScanner(telegram_token, telegram_chat_id, exchange, timeframe, offset)
    results = await scanner.scan_all_markets()
    
    if results:
        print(f"\nFound {len(results)} Confluence patterns:")
        
        # Convert results to DataFrame for console display
        df_results = pd.DataFrame(results)
        
        # Round numeric columns
        df_results['volume_usd'] = df_results['volume_usd'].round(2)
        df_results['close'] = df_results['close'].round(8)
        df_results['volume'] = df_results['volume'].round(2)
        df_results['volume_ratio'] = df_results['volume_ratio'].round(2)
        df_results['close_off_low'] = df_results['close_off_low'].round(1)
        df_results['momentum_score'] = df_results['momentum_score'].round(4)
        
        # Reorder columns for better display
        display_cols = ['symbol', 'close', 'volume_usd', 'volume_ratio', 'close_off_low', 
                       'momentum_score', 'high_volume', 'spread_breakout', 'momentum_breakout']
        available_cols = [col for col in display_cols if col in df_results.columns]
        
        # Display the results
        print(df_results[available_cols])
        
        # Show component analysis
        print(f"\n🔧 COMPONENT ANALYSIS:")
        vol_count = df_results['high_volume'].sum()
        spread_count = df_results['spread_breakout'].sum()
        momentum_count = df_results['momentum_breakout'].sum()
        
        print(f"High Volume signals: {vol_count}/{len(results)} ({vol_count/len(results)*100:.1f}%)")
        print(f"Spread Breakout signals: {spread_count}/{len(results)} ({spread_count/len(results)*100:.1f}%)")
        print(f"Momentum Breakout signals: {momentum_count}/{len(results)} ({momentum_count/len(results)*100:.1f}%)")
        
    else:
        print(f"\nNo Confluence patterns found in {offset_desc}")

# Set up logging
logging.basicConfig(level=logging.ERROR)

# Apply nest_asyncio to allow async operations in Jupyter
nest_asyncio.apply()

# Example usage functions
async def scan_current_confluence():
    """Scan current candle for confluence"""
    await run_confluence_scanner("Kucoin", "1w", offset=0)

async def scan_closed_confluence():
    """Scan last closed candle for confluence"""
    await run_confluence_scanner("Kucoin", "1w", offset=1)

async def scan_previous_confluence():
    """Scan two candles ago for confluence"""
    await run_confluence_scanner("Kucoin", "1w", offset=2)

# Main execution function
async def main():
    """
    Main execution - modify parameters here
    """
    exchange = "Mexc"  # Binance, Kucoin, Mexc
    timeframe = "1w"     # 1d, 2d, 1w
    offset = 0           # 0 = current candle, 1 = last closed candle, 2 = two candles ago
    
    await run_confluence_scanner(exchange, timeframe, offset)

# Run the async main function
print("🔍 CONFLUENCE SCANNER")
print("=" * 30)
print("Available functions:")
print("• await main() - Run with default settings")
print("• await scan_current_confluence() - Scan current candle")
print("• await scan_closed_confluence() - Scan last closed candle")
print("• await scan_previous_confluence() - Scan two candles ago")
print("• await run_confluence_scanner('Exchange', 'timeframe', offset) - Custom scan")
print("\nExample: await main()")

# Uncomment to auto-run:
await main()

In [ ]:
# Direct Test Bar Pattern Tester
# This script directly tests the calculate_test_bar function

import sys
import os
import logging
import pandas as pd
import numpy as np
import nest_asyncio
import asyncio
from tqdm.notebook import tqdm

# Apply nest_asyncio to make asyncio work in Jupyter
nest_asyncio.apply()

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Add project directory to path
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)
print(f"✓ Added {project_dir} to sys.path")

# Import the necessary modules
from exchanges import MexcSpotClient, BybitSpotClient, KucoinSpotClient

def calculate_test_bar_direct(df):
    """
    Direct implementation of Test Bar pattern detection for testing
    """
    df = df.copy()
    
    # Calculate basic metrics
    df['bar_range'] = df['high'] - df['low']
    
    # Function to check if current bar is an inside bar
    df['is_inside_bar'] = (df['high'] < df['high'].shift(1)) & (df['low'] > df['low'].shift(1))
    
    # Function to check if bar is down (close < open)
    df['is_down_bar'] = df['close'] < df['open']
    
    # Function to check if close is in higher 65% of the spread (closing off the lows)
    df['close_position'] = np.where(df['bar_range'] != 0, (df['close'] - df['low']) / df['bar_range'], 0)
    df['close_off_lows'] = df['close_position'] >= 0.65
    
    # Function to check if current volume is less than 40% of previous bar volume
    df['lower_volume_than_prev'] = df['volume'] < (df['volume'].shift(1) * 0.4)
    
    # Function to check if current volume is the lowest in last 3 bars
    df['lowest_volume_in_3_bars'] = (df['volume'] < df['volume'].shift(1)) & (df['volume'] < df['volume'].shift(2))
    
    # Debug columns
    df['debug_inside'] = df['is_inside_bar']
    df['debug_down'] = df['is_down_bar'] 
    df['debug_close_off_lows'] = df['close_off_lows']
    df['debug_lower_vol'] = df['lower_volume_than_prev']
    df['debug_lowest_vol_3'] = df['lowest_volume_in_3_bars']
    
    # Main condition: all criteria must be met
    test_bar_pattern = (
        df['is_inside_bar'] &
        df['is_down_bar'] &
        df['close_off_lows'] &
        df['lower_volume_than_prev'] &
        df['lowest_volume_in_3_bars']
    )
    
    # Signal only new occurrences
    test_bar = test_bar_pattern & ~test_bar_pattern.shift(1).fillna(False)
    
    return test_bar, df

async def test_direct_test_bar(exchange_client_class, timeframe, symbol):
    """Test the direct test_bar function on a specific symbol"""
    print(f"Testing DIRECT Test Bar detection on {symbol} ({timeframe})")
    
    # Initialize exchange client and fetch data
    client = exchange_client_class(timeframe=timeframe)
    try:
        await client.init_session()
        df = await client.fetch_klines(symbol)
    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        return
    finally:
        await client.close_session()
    
    if df is None or len(df) < 10:
        print(f"No data fetched for {symbol} or insufficient data")
        return
    
    print(f"\\nLast 5 bars for {symbol}:")
    print(df[['open', 'high', 'low', 'close', 'volume']].tail(5))
    
    # Apply direct test_bar function
    print("\\nApplying direct test_bar detection...")
    try:
        test_bar_signals, df_with_debug = calculate_test_bar_direct(df)
        
        # Quick results first
        current_detected = test_bar_signals.iloc[-1]
        prev_detected = test_bar_signals.iloc[-2] if len(df) > 1 else False
        
        print("\\n" + "="*60)
        print("QUICK RESULTS")
        print("="*60)
        print(f"Current Bar Test Pattern:  {current_detected}")
        print(f"Previous Bar Test Pattern: {prev_detected}")
        
        # If pattern detected, show which bar(s)
        if current_detected or prev_detected:
            print("\\nPATTERN DETECTED!")
            if current_detected:
                print("  -> Current bar matches test pattern")
            if prev_detected:
                print("  -> Previous bar matches test pattern")
        else:
            print("\\nNO PATTERN DETECTED")
        
        print("\\n" + "="*60)
        print("CONDITION BREAKDOWN")
        print("="*60)
        
        # Show only last 2 bars for condition analysis
        for i in range(max(0, len(df)-2), len(df)):
            if i < 2:  # Skip early bars that don't have enough history
                continue
            
            bar_range = df['high'].iloc[i] - df['low'].iloc[i]
            close_position = (df['close'].iloc[i] - df['low'].iloc[i]) / bar_range * 100 if bar_range > 0 else 0
            
            print(f"\\nBAR {i} ({df.index[i].date()})")
            print(f"OHLC: O={df['open'].iloc[i]:.6f} H={df['high'].iloc[i]:.6f} L={df['low'].iloc[i]:.6f} C={df['close'].iloc[i]:.6f}")
            print(f"Volume: {df['volume'].iloc[i]:,.0f}")
            
            if i > 0:
                prev_high = df['high'].iloc[i-1]
                prev_low = df['low'].iloc[i-1]
                prev_volume = df['volume'].iloc[i-1]
                volume_40_pct = prev_volume * 0.4
                
                # Show conditions in a clean table format
                conditions = [
                    ("Inside Bar", df_with_debug['debug_inside'].iloc[i]),
                    ("Down Bar", df_with_debug['debug_down'].iloc[i]),
                    ("Close >= 65% from Low", df_with_debug['debug_close_off_lows'].iloc[i]),
                    ("Volume < 40% of Previous", df_with_debug['debug_lower_vol'].iloc[i]),
                    ("Lowest Volume in 3 bars", df_with_debug['debug_lowest_vol_3'].iloc[i])
                ]
                
                print("\\nConditions:")
                for condition_name, result in conditions:
                    status = "✓" if result else "✗"
                    print(f"  {status} {condition_name:<25} {result}")
                
                # Show key metrics
                print("\\nKey Metrics:")
                print(f"  Close Position: {close_position:.1f}% from low")
                print(f"  Current Volume: {df['volume'].iloc[i]:,.0f}")
                print(f"  40% of Prev Vol: {volume_40_pct:,.0f}")
                
                if i > 1:
                    vol_prev_2 = df['volume'].iloc[i-2]
                    print(f"  Previous Vol: {prev_volume:,.0f}")
                    print(f"  Previous-2 Vol: {vol_prev_2:,.0f}")
                
                # Final result for this bar
                all_conditions_met = all(result for _, result in conditions)
                print(f"\\nAll Conditions Met: {all_conditions_met}")
                print(f"Test Bar Signal: {test_bar_signals.iloc[i]}")
                print("-" * 50)
        
    except Exception as e:
        print(f"Error running direct test_bar detection: {str(e)}")
        import traceback
        traceback.print_exc()

# Test parameters
exchange_client = KucoinSpotClient
timeframe = "1d"
symbol = "TEL-USDT"

# Run the test
await test_direct_test_bar(exchange_client, timeframe, symbol)

print("\\n✅ Direct test bar detection test completed")

In [ ]:
"""
Standalone HLC Bar Chart Plotter
A reusable function for plotting HLC bars with optional pattern highlighting
"""

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from matplotlib.lines import Line2D

def plot_hlc_bars(data, highlighted_bars=None, title="HLC Chart", symbol="SYMBOL", 
                  interval="1d", semilog=False, highlight_color="fuchsia", 
                  highlight_label="Pattern", figsize=(14, 10), show_volume=True):
    """
    Plot HLC bar chart with optional pattern highlighting
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with columns: ['datetime', 'high', 'low', 'close', 'volume']
        - datetime: timestamp column (will be used for x-axis)
        - high: high prices
        - low: low prices  
        - close: close prices
        - volume: volume data (optional if show_volume=False)
        
    highlighted_bars : pandas.Series or list/array, optional
        Boolean series or array indicating which bars to highlight
        Length must match data length
        
    title : str, default "HLC Chart"
        Chart title
        
    symbol : str, default "SYMBOL" 
        Symbol name for display
        
    interval : str, default "1d"
        Time interval for date formatting (1m, 5m, 15m, 30m, 1h, 4h, 1d, 3d, 1w, 1M)
        
    semilog : bool, default False
        Use logarithmic scale for price chart
        
    highlight_color : str, default "fuchsia"
        Color for highlighted bars
        
    highlight_label : str, default "Pattern"
        Label for highlighted bars in legend
        
    figsize : tuple, default (14, 10)
        Figure size (width, height)
        
    show_volume : bool, default True
        Whether to show volume subplot
        
    Returns:
    --------
    matplotlib.figure.Figure
        The created figure object
    """
    
    # Input validation
    if not isinstance(data, pd.DataFrame):
        raise ValueError("data must be a pandas DataFrame")
    
    # Check required columns
    required_cols = ['datetime', 'high', 'low', 'close']
    if show_volume:
        required_cols.append('volume')
    
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    if len(data) == 0:
        raise ValueError("data DataFrame is empty")
    
    # Validate highlighted_bars
    if highlighted_bars is not None:
        if len(highlighted_bars) != len(data):
            raise ValueError("highlighted_bars length must match data length")
        # Convert to boolean array
        highlighted_bars = np.array(highlighted_bars, dtype=bool)
    else:
        highlighted_bars = np.zeros(len(data), dtype=bool)
    
    # Create figure with subplots
    if show_volume:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, 
                                       gridspec_kw={'height_ratios': [3, 1]})
    else:
        fig, ax1 = plt.subplots(1, 1, figsize=figsize)
        ax2 = None
    
    # Convert dates to numbers for plotting
    dates = mdates.date2num(data['datetime'])
    
    # Calculate margins and tick length
    date_range = dates[-1] - dates[0] if len(dates) > 1 else 1
    margin = date_range * 0.05
    
    # Calculate actual bar spacing for consistent tick length
    if len(dates) > 1:
        avg_bar_spacing = date_range / (len(dates) - 1)
        tick_length = avg_bar_spacing * 0.4  # 40% of bar spacing
        volume_bar_width = avg_bar_spacing * 0.8
    else:
        tick_length = date_range * 0.01
        volume_bar_width = date_range * 0.02
    
    # Draw HLC bars
    for i, (date, high, low, close) in enumerate(zip(dates, data['high'], data['low'], data['close'])):
        is_highlighted = highlighted_bars[i]
        color = highlight_color if is_highlighted else 'black'
        line_width = 1.2
        
        # Vertical line from low to high
        ax1.plot([date, date], [low, high], color=color, linewidth=line_width, solid_capstyle='butt')
        
        # Horizontal tick mark for close (on the right side)
        ax1.plot([date, date + tick_length], [close, close], color=color, 
                linewidth=line_width+0.5, solid_capstyle='butt')
    
    # Configure price chart
    if semilog:
        ax1.set_yscale('log')
        scale_text = "Semilog Scale"
    else:
        scale_text = "Linear Scale"
    
    ax1.set_title(f'{symbol} {title} - {scale_text}', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Price', fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Format x-axis based on timeframe
    _format_datetime_axis(ax1, interval)
    ax1.set_xlim(dates[0] - margin, dates[-1] + margin)
    
    # Volume chart
    if show_volume and ax2 is not None:
        volume_colors = [highlight_color if highlighted_bars[i] else 'orange' for i in range(len(dates))]
        volume_edges = ['darkmagenta' if highlighted_bars[i] else 'darkorange' for i in range(len(dates))]
        
        ax2.bar(dates, data['volume'], width=volume_bar_width, alpha=0.6, 
                color=volume_colors, edgecolor=volume_edges)
        ax2.set_ylabel('Volume', fontsize=12)
        ax2.set_xlabel('Date', fontsize=12)
        ax2.grid(True, alpha=0.3)
        
        _format_datetime_axis(ax2, interval)
        ax2.set_xlim(dates[0] - margin, dates[-1] + margin)
        plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
    else:
        ax1.set_xlabel('Date', fontsize=12)
    
    # Create legend
    legend_elements = [
        Line2D([0], [0], color='black', linewidth=2, label='High-Low Range'),
        Line2D([0], [0], color='black', linewidth=3, label='Close Price (right tick)')
    ]
    
    # Add highlighted bars to legend if any exist
    if highlighted_bars.any():
        legend_elements.append(
            Line2D([0], [0], color=highlight_color, linewidth=3, label=highlight_label)
        )
        highlight_count = highlighted_bars.sum()
    else:
        highlight_count = 0
    
    ax1.legend(handles=legend_elements, loc='upper left', title=scale_text)
    
    # Add pattern count if patterns exist
    if highlight_count > 0:
        ax1.text(0.99, 0.95, f'{highlight_label}: {highlight_count}', 
                transform=ax1.transAxes, ha='right', va='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                fontsize=10)
    
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    plt.tight_layout()
    
    return fig

def _format_datetime_axis(ax, interval):
    """Helper function to format datetime axis based on interval"""
    if interval in ['1m', '5m', '15m', '30m']:
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=6))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))
    elif interval in ['1h', '2h', '4h', '6h', '12h']:
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    elif interval in ['1d', '3d']:
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    else:  # weekly, monthly
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Example usage and test function
def example_usage():
    """Example showing how to use the plot_hlc_bars function"""
    
    # Create sample data
    import datetime
    dates = pd.date_range('2023-01-01', periods=100, freq='D')
    np.random.seed(42)
    
    # Generate realistic OHLC data
    closes = 100 + np.cumsum(np.random.randn(100) * 0.02)
    highs = closes + np.random.rand(100) * 5
    lows = closes - np.random.rand(100) * 5
    volumes = np.random.rand(100) * 1000000
    
    data = pd.DataFrame({
        'datetime': dates,
        'high': highs,
        'low': lows, 
        'close': closes,
        'volume': volumes
    })
    
    # Create some random pattern detections
    pattern_detected = np.random.choice([True, False], size=100, p=[0.1, 0.9])
    
    # Plot with pattern highlighting
    fig = plot_hlc_bars(
        data=data,
        highlighted_bars=pattern_detected,
        title="Daily Chart with Pattern Detection",
        symbol="EXAMPLE",
        interval="1d",
        semilog=False,
        highlight_color="red",
        highlight_label="Detected Pattern"
    )
    
    plt.show()
    return fig

# Scanner integration example
def scanner_integration_example():
    """Example of how to integrate with a scanner function"""
    
    def my_pattern_scanner(data):
        """
        Example scanner function - replace with your actual scanner logic
        Returns boolean array indicating pattern detection
        """
        # Example: detect when close > 20-period moving average
        ma20 = data['close'].rolling(20).mean()
        pattern = (data['close'] > ma20) & (data['volume'] > data['volume'].rolling(10).mean())
        return pattern.fillna(False)
    
    # Your data loading logic here
    # data = load_your_data()  # Replace with actual data loading
    
    # For demo, create sample data
    dates = pd.date_range('2023-01-01', periods=200, freq='D')
    np.random.seed(42)
    closes = 100 + np.cumsum(np.random.randn(200) * 0.02)
    highs = closes + np.random.rand(200) * 3
    lows = closes - np.random.rand(200) * 3
    volumes = np.random.rand(200) * 1000000
    
    data = pd.DataFrame({
        'datetime': dates,
        'high': highs,
        'low': lows,
        'close': closes,
        'volume': volumes
    })
    
    # Run your scanner
    detected_patterns = my_pattern_scanner(data)
    
    # Plot only if patterns are detected
    if detected_patterns.any():
        print(f"Patterns detected! Found {detected_patterns.sum()} occurrences")
        fig = plot_hlc_bars(
            data=data,
            highlighted_bars=detected_patterns,
            title="Scanner Results",
            symbol="SCANNED_SYMBOL",
            interval="1d",
            highlight_color="lime",
            highlight_label="Scanner Hit"
        )
        plt.show()
    else:
        print("No patterns detected")

if __name__ == "__main__":
    # Run example
    example_usage()